In [1]:
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr
import glob
from scipy.stats import norm
from scipy import stats
from pandas import Series,DataFrame

# find the nearest grid

In [3]:
obs_t2 = pd.read_csv('observation_mean.csv')
obs_t2 = obs_t2.rename(columns={"mean": "obs_mean"})

In [5]:
#use one wrfout file as an reference
ncfile = Dataset("wrfout_d04_2011-06-23_11:00:00")

In [6]:
#find the nearest grid dell to the observation station
nearest_grid = ll_to_xy(ncfile,obs_t2['lat'],obs_t2['lon'])
nearest_grid_df = nearest_grid.to_dataframe()

In [8]:
x = nearest_grid[0,:]
x_df = x.to_dataframe().reset_index()
x_df = x_df['xy']
x_df = pd.DataFrame(x_df)

y = nearest_grid[1,:]
y_df = y.to_dataframe().reset_index()
y_df = y_df['xy']
y_df = pd.DataFrame(y_df)

obs_t2['west_east'] = x_df['xy']
obs_t2['south_north'] = y_df['xy']

# ERA5_t2

In [11]:
ds_era5_t2 = xr.open_dataset('t2_1620_era5.nc')
t2_1620_era5 = ds_era5_t2['t2_1620_era5']

In [12]:
t2_1620_era5 = t2_1620_era5.mean(dim='Time')

In [13]:
t2_1620_era5_df = t2_1620_era5.to_dataframe().reset_index()

In [14]:
obs_t2 = pd.merge(obs_t2,t2_1620_era5_df,on=['west_east','south_north'],how="left")

In [15]:
del obs_t2['XLONG']
del obs_t2['XLAT']

# CMIP6_t2

In [16]:
ds_cmip6_t2 = xr.open_dataset('t2_1620_ssp245.nc')
t2_1620_cmip6 = ds_cmip6_t2_2['t2_1620_ssp245']

In [17]:
t2_1620_cmip6 = t2_1620_cmip6.mean(dim='Time')

In [18]:
t2_1620_cmip6_df = t2_1620_cmip6.to_dataframe().reset_index()

In [19]:
obs_t2 = pd.merge(obs_t2,t2_1620_cmip6_df,on=['west_east','south_north'],how="left")

In [20]:
obs_t2 = obs_t2.rename(columns={"t2_1620_ssp245": "t2_1620_cmip6"})

In [21]:
del obs_t2['XLONG']
del obs_t2['XLAT']

# t2_1620_corrected_cmip6

In [28]:
ds_bias_cmip6_t2_corrected = xr.open_dataset('t2_1620_era1120_corrected.nc')
t2_1620_corrected_cmip6 = ds_bias_cmip6_t2_corrected['t2_1620_era51120_corrected']

In [29]:
t2_1620_corrected_cmip6 = t2_1620_corrected_cmip6.mean(dim='Time')

In [30]:
t2_1620_corrected_cmip6_df = t2_1620_corrected_cmip6.to_dataframe().reset_index()

In [31]:
obs_t2 = pd.merge(obs_t2,t2_1620_corrected_cmip6_df,on=['west_east','south_north'],how="left")

In [32]:
del obs_t2['XLONG']
del obs_t2['XLAT']

# process obs_t2

In [34]:
obs_t2.dropna(axis=0,how='any',inplace=True)

obs_t2 = obs_t2.reset_index(drop=True)

In [36]:
del obs_t2['LU_INDEX_x']
del obs_t2['LU_INDEX_y']

In [37]:
obs_t2.to_csv('obs_station_withsimu_value.csv',index=False)

# Mean Bias between datasets

In [40]:
obs_t2['t2_1620_era5'] = obs_t2['t2_1620_era5']-obs_t2['obs_mean']
obs_t2['t2_1620_cmip6'] = obs_t2['t2_1620_cmip6']-obs_t2['obs_mean']
obs_t2['t2_1620_era51120_corrected'] = obs_t2['t2_1620_era51120_corrected']-obs_t2['obs_mean']

In [41]:
obs_t2.to_csv('bias_correction_diff.csv',index=False)